In [3]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate

# Establish the database connection
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='hospital'
)

cursor = connection.cursor()

# Check if the connection is successful
if connection.is_connected():
    print("Connection opened successfully.")


def fetch_all_records(query):
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")
        return []


def display_doctor_list():
    query = "SELECT * FROM DOCTOR"
    results = fetch_all_records(query)

    print("\nDoctor List:")
    if results:
        headers = [desc[0] for desc in cursor.description]
        print(tabulate(results, headers=headers, tablefmt="pretty"))
    else:
        print("No records found in the DOCTOR table.")


def display_patient_list():
    query = "SELECT * FROM PATIENT"
    results = fetch_all_records(query)

    print("\nPatient List:")
    if results:
        headers = [desc[0] for desc in cursor.description]
        print(tabulate(results, headers=headers, tablefmt="pretty"))
    else:
        print("No records found in the PATIENT table.")


def display_doctor_history():
    query = "SELECT * FROM APPOINT"
    results = fetch_all_records(query)

    print("\nDoctor History:")
    if results:
        headers = [desc[0] for desc in cursor.description]
        print(tabulate(results, headers=headers, tablefmt="pretty"))
    else:
        print("No records found in the APPOINT table.")


def display_patient_history():
    query = """
        SELECT APPOINT.OPDNO, APPOINT.DOA, PATIENT.PNAME, PATIENT.PADD, PATIENT.PMOB, APPOINT.DNAME, APPOINT.SYMPTOM, APPOINT.TREATMENT
        FROM APPOINT
        JOIN PATIENT ON APPOINT.PNAME = PATIENT.PNAME;
    """
    results = fetch_all_records(query)

    print("\nPatient History:")
    if results:
        headers = [desc[0] for desc in cursor.description]
        print(tabulate(results, headers=headers, tablefmt="pretty"))
    else:
        print("No records found in the APPOINT and PATIENT tables.")


def display_opd_details():
    print("The OPD details:")

    try:
        beginning_date = input("Enter the starting date in YYYY-MM-DD: ")
        end_date = input("Enter the ending date in YYYY-MM-DD: ")

        # Validate the date format
        if not (beginning_date and end_date):
            print("Invalid date format. Please use YYYY-MM-DD.")
            return

        query = """
            SELECT APPOINT.OPDNO, APPOINT.DOA, APPOINT.PNAME, APPOINT.DNAME, APPOINT.SYMPTOM, APPOINT.TREATMENT, OPD.FEE
            FROM APPOINT JOIN OPD ON APPOINT.OPDNO = OPD.OPDNO WHERE APPOINT.DOA BETWEEN %s AND %s
        """
        cursor.execute(query, (beginning_date, end_date))
        results = cursor.fetchall()

        if results:
            headers = ["opdno", "doa", "pname", "dname", "symptom", "treatment", "fee"]
            print(tabulate(results, headers=headers, tablefmt="pretty"))
        else:
            print("No records found.")

    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")

    except Exception as e:
        print(f"An error occurred: {e}")


def display_max_diagnosis_chart():
    print("Maximum Diagnose:")

    try:
        query = "SELECT symptom, COUNT(symptom) AS total_cases FROM appoint GROUP BY symptom;"
        df = pd.read_sql(query, connection)

        if not df.empty:
            print(tabulate(df, headers="keys", tablefmt="pretty"))

            # Set width, height, and color for the bar chart
            bar_width = 0.5
            bar_height = df['total_cases']
            bar_color = 'skyblue'

            plt.bar(df['symptom'], bar_height, width=bar_width, color=bar_color)
            plt.xlabel("Symptom")
            plt.ylabel("Total Cases")
            plt.title("Diagnose Chart")
            plt.show()
        else:
            print("No records found.")

    except ValueError as val_err:
        print(f"Invalid input: {val_err}")

    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")

    except Exception as e:
        print(f"Unknown error: {e}")


Connection opened successfully.
